In [21]:
# ============================================================================
# RETAILROCKET DATASET - EXPLORATORY DATA ANALYSIS
# ============================================================================
# Dataset: E-commerce user behavior (views, add-to-cart, purchases)
# Goal: Understand user patterns, item popularity, and conversion funnels
# ============================================================================

In [22]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [23]:
# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [24]:
# ============================================================================
# 1. PROJECT SETUP
# ============================================================================

In [25]:
# Setup project paths (portable & reusable)
# - Prints current working directory (for debugging)
# - Defines PROJECT_ROOT as the main project folder
# - Builds a reusable DATA_DIR path to raw data

cwd = Path(os.getcwd())
if (cwd / "data").exists():
    PROJECT_ROOT = cwd
else:
    PROJECT_ROOT = cwd.parent  
print("project root:", PROJECT_ROOT)

DATA_DIR = PROJECT_ROOT / "data" / "raw" / "retailrocket"
print("data dir:", DATA_DIR)
print("files:", list(DATA_DIR.glob("*"))[:10])  

project root: c:\Users\yasmi\projects\dynamic-recommendation-system
data dir: c:\Users\yasmi\projects\dynamic-recommendation-system\data\raw\retailrocket
files: [WindowsPath('c:/Users/yasmi/projects/dynamic-recommendation-system/data/raw/retailrocket/category_tree.csv'), WindowsPath('c:/Users/yasmi/projects/dynamic-recommendation-system/data/raw/retailrocket/events.csv'), WindowsPath('c:/Users/yasmi/projects/dynamic-recommendation-system/data/raw/retailrocket/item_properties_part1.csv'), WindowsPath('c:/Users/yasmi/projects/dynamic-recommendation-system/data/raw/retailrocket/item_properties_part2.csv')]


In [26]:
# ============================================================================
# 2. DATA LOADING
# ============================================================================

In [27]:
events = pd.read_csv(DATA_DIR / "events.csv",dtype={'timestamp': 'int64'})
category_tree = pd.read_csv(DATA_DIR / "category_tree.csv")
prop1 = pd.read_csv(DATA_DIR / "item_properties_part1.csv",dtype={'timestamp': 'int64'})
prop2 = pd.read_csv(DATA_DIR / "item_properties_part2.csv",dtype={'timestamp': 'int64'})

In [28]:
# Merging properties
item_properties = pd.concat([prop1, prop2], ignore_index=True)

In [29]:
print(f"Item properties merged: {item_properties.shape} (part1: {prop1.shape}, part2: {prop2.shape})")


Item properties merged: (20275902, 4) (part1: (10999999, 4), part2: (9275903, 4))


In [30]:
# ============================================================================
# 3. INITIAL DATA OVERVIEW
# ============================================================================

In [31]:
datasets = {
    'events': events,
    'category_tree': category_tree,
    'item_properties': item_properties
}

for name, df in datasets.items():
    print(f"\n{name.upper()}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
    print(f"\nFirst 3 rows:")
    print(df.head(3).to_string(index=False))
    print(f"Nulls:\n{df.isna().sum()}")


EVENTS
Shape: (2756101, 5)
Columns: ['timestamp', 'visitorid', 'event', 'itemid', 'transactionid']
Memory: 256.82 MB

First 3 rows:
    timestamp  visitorid event  itemid  transactionid
1433221332117     257597  view  355908            NaN
1433224214164     992329  view  248676            NaN
1433221999827     111016  view  318965            NaN
Nulls:
timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

CATEGORY_TREE
Shape: (1669, 2)
Columns: ['categoryid', 'parentid']
Memory: 0.03 MB

First 3 rows:
 categoryid  parentid
       1016     213.0
        809     169.0
        570       9.0
Nulls:
categoryid     0
parentid      25
dtype: int64

ITEM_PROPERTIES
Shape: (20275902, 4)
Columns: ['timestamp', 'itemid', 'property', 'value']
Memory: 3067.60 MB

First 3 rows:
    timestamp  itemid   property                           value
1435460400000  460429 categoryid                            1338
144150840

In [32]:
# ============================================================================
# 4. EVENTS - PREPROCESSING & ANALYSIS
# ============================================================================

In [33]:
# Convert timestamp
events['timestamp'] = pd.to_datetime(events['timestamp'], unit='ms')
events['date'] = events['timestamp'].dt.date
events['hour'] = events['timestamp'].dt.hour
events['dayofweek'] = events['timestamp'].dt.dayofweek
events['day_name'] = events['timestamp'].dt.day_name()

In [34]:
item_col ="itemid"
user_col = "visitorid"
event_col = "event" 

# unique counts
print("\nunique counts (sample):")
for c in [user_col, item_col, event_col]:
    if c in events.columns:
        print(c, "unique:", events[c].nunique())


# top products and top users in the sample
print("\nTop 10 items (by events):\n", events[item_col].value_counts().head(10))
print("\nTop 10 users (by events):\n", events[user_col].value_counts().head(10))


unique counts (sample):
visitorid unique: 1407580
itemid unique: 235061
event unique: 3

Top 10 items (by events):
 itemid
187946    3412
461686    2978
5411      2334
370653    1854
219512    1800
257040    1647
298009    1642
96924     1633
309778    1628
384302    1608
Name: count, dtype: int64

Top 10 users (by events):
 visitorid
1150086    7757
530559     4328
152963     3024
895999     2474
163561     2410
371606     2345
286616     2252
684514     2246
892013     2024
861299     1991
Name: count, dtype: int64


In [35]:
#Basic Statistics
print(f"\nBasic Statistics:")
print(f"  Total events: {len(events):,}")
print(f"  Unique visitors: {events['visitorid'].nunique():,}")
print(f"  Unique items: {events['itemid'].nunique():,}")
print(f"  Date range: {events['timestamp'].min()} to {events['timestamp'].max()}")
print(f"  Days span: {(events['timestamp'].max() - events['timestamp'].min()).days} days")


Basic Statistics:
  Total events: 2,756,101
  Unique visitors: 1,407,580
  Unique items: 235,061
  Date range: 2015-05-03 03:00:04.384000 to 2015-09-18 02:59:47.788000
  Days span: 137 days


In [36]:
print("\nEvent Type Distribution:")
event_dist = events['event'].value_counts()
for event_type, count in event_dist.items():
    pct = count / len(events) * 100
    print(f"  {event_type}: {count:,} ({pct:.2f}%)")


Event Type Distribution:
  view: 2,664,312 (96.67%)
  addtocart: 69,332 (2.52%)
  transaction: 22,457 (0.81%)


In [37]:
# Conversion metrics
total_views = event_dist.get('view', 0)
total_addtocart = event_dist.get('addtocart', 0)
total_transactions = event_dist.get('transaction', 0)

print("\nConversion Funnel:")
print(f"  Views → AddToCart: {total_addtocart/total_views*100:.2f}%" if total_views > 0 else "N/A")
print(f"  Views → Purchase: {total_transactions/total_views*100:.2f}%" if total_views > 0 else "N/A")
print(f"  AddToCart → Purchase: {total_transactions/total_addtocart*100:.2f}%" if total_addtocart > 0 else "N/A")


Conversion Funnel:
  Views → AddToCart: 2.60%
  Views → Purchase: 0.84%
  AddToCart → Purchase: 32.39%


In [38]:
print(events['timestamp'].min())
print(events['timestamp'].max())

2015-05-03 03:00:04.384000
2015-09-18 02:59:47.788000


In [39]:
# ============================================================================
# 5. ITEM PROPERTIES - analysis
# ============================================================================


In [40]:
# Convert timestamp
item_properties['timestamp'] = pd.to_datetime(item_properties['timestamp'], unit='ms')


In [43]:
print(f"\nBasic Statistics:")
print(f"  Total property records: {len(item_properties):,}")
print(f"  Unique items: {item_properties['itemid'].nunique():,}")
print(f"  Unique properties: {item_properties['property'].nunique()}")
print(f"  Date range: {item_properties['timestamp'].min()} to {item_properties['timestamp'].max()}")



Basic Statistics:
  Total property records: 20,275,902
  Unique items: 417,053
  Unique properties: 1104
  Date range: 2015-05-10 03:00:00 to 2015-09-13 03:00:00


In [46]:
print("\n📋 Property Types Overview:")
prop_counts = item_properties['property'].value_counts()
print(f"  Total unique property types: {len(prop_counts):,}")

# Show only the most common properties (top 10)
print(f"\n  Top 10 Most Common Properties:")
for prop, count in prop_counts.head(10).items():
    unique_items = item_properties[item_properties['property'] == prop]['itemid'].nunique()
    print(f"    {prop}: {count:,} records across {unique_items:,} items")



📋 Property Types Overview:
  Total unique property types: 1,104

  Top 10 Most Common Properties:
    888: 3,000,398 records across 417,053 items
    790: 1,790,516 records across 417,053 items
    available: 1,503,639 records across 417,053 items
    categoryid: 788,214 records across 417,053 items
    6: 631,471 records across 409,065 items
    283: 597,419 records across 417,053 items
    776: 574,220 records across 407,305 items
    678: 481,966 records across 417,019 items
    364: 476,486 records across 417,053 items
    202: 448,938 records across 414,217 items


In [50]:
# Property coverage per item
print("\n📊 Property Coverage per Item:")
props_per_item = item_properties.groupby('itemid')['property'].nunique()
print(f"  Items with 1 property: {(props_per_item == 1).sum():,}")
print(f"  Items with 2+ properties: {(props_per_item >= 2).sum():,}")
print(f"  Items with 5+ properties: {(props_per_item >= 5).sum():,}")



📊 Property Coverage per Item:
  Items with 1 property: 0
  Items with 2+ properties: 417,053
  Items with 5+ properties: 417,053


In [51]:
# Sample item with all properties
print("\n🔍 Sample Item Properties:")
# Find an item with many properties but not the absolute max (which might be an outlier)
props_per_item_sorted = item_properties.groupby('itemid')['property'].nunique().sort_values(ascending=False)
sample_item = props_per_item_sorted.iloc[min(10, len(props_per_item_sorted)-1)]  # 11th item or last
sample_item_id = props_per_item_sorted.index[min(10, len(props_per_item_sorted)-1)]
sample_props = item_properties[item_properties['itemid'] == sample_item_id][['property', 'value']].head(15)
print(f"Item {sample_item_id} (has {sample_item} properties, showing first 15):")
print(sample_props.to_string(index=False))
if sample_item > 15:
    print(f"  ... and {sample_item - 15} more properties")


🔍 Sample Item Properties:
Item 406081 (has 56 properties, showing first 15):
 property                         value
      355                        586893
      523 991263 1238769 126079 1141242
available                             0
      678                        367680
      981    288796 23770 769062 769062
      981    288796 23770 769062 769062
      701                        769062
      765                        769062
      981           288796 23770 769062
      112                        679677
      227                       1249027
      447               1107648 1176545
      888                 565165 367680
      815                        769062
      888                565165 1154859
  ... and 41 more properties
